# Automated Multiple Reaction Monitoring (MRM)-profiling and Ozone Electrospray Ionizaton (OzESI)-MRM Informatics Platform for High-throughput Lipidomics


In this jupyter notebook you will automate the data analysis of the lipidome. This is a challenging problem to perform manually due to the diverse nature of lipids and the many potential isomers. In this notebook you will analyze mzML files containing data from lipid MRMs, with ozone off and ozone on. The goal is to identify possible double-bond locations in a lipid, in this case a TAG (triacylglycerols).

In [1]:
from IPython.display import Image

![title](Figures/agilent_lcms.png)

The examples shown here were run on an Agilent 6495C Triple Quadrupole LC/MS (example shown above) that has been connected to an ozone line (not shown in picture) for ozoneolysis of lipids.

![title](Figures/TAG_example.png)
Here is an example of a TAG. Notice how many possibilities there are for locations of one double-bond there could be and how convoluted the analysis can become! This image is obtained from LipidMaps.org

Import all necessary libraries

In [2]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px

No module named 'ms_deisotope._c.averagine' averagine
No module named 'ms_deisotope._c.scoring'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'


KeyboardInterrupt: 

In [ ]:
#Make a list of all folder names in data_mzml
folder_names_data_mzml = os.listdir('data_mzml')
print(folder_names_data_mzml)
#Make a list of all folder names in data_mzml/fly_1-11-23
folder_name = os.listdir('data_mzml/03-03-23')
print(folder_name)


MAKE CLASSES FOR EACH LIPID

In [ ]:
mrm_list_new = pd.read_excel('SUPPLE_2.XLS')
lipid_types = ["CE","TAG","CER","FFA","PC","PE","PG","PI","SM","AC"]

#loop through all sheets in SUPPLE_2.XLS and make a df of Compound Name, Parent Ion, and Product Ion
mrm_list_new = pd.read_excel('SUPPLE_2.XLS', sheet_name = None)
mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion']]
#Add underscore to middle of columns names
mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
#round Parent Ion and Product Ion to 1 decimal place
mrm_list_offical['Parent_Ion'] = np.floor(mrm_list_offical['Parent_Ion'].round(1))
mrm_list_offical['Product_Ion'] = np.floor(mrm_list_offical['Product_Ion'].round(1))
#create transition column by combining Parent Ion and Product Ion with arrow between numbers
mrm_list_offical['Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
#change column compound name to lipid
mrm_list_offical = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
#make a column called Class match lipid column to lipid types


mrm_list_offical['Class'] = mrm_list_offical['Lipid'].str.extract('([a-zA-Z]+)')
#if Class = O or omega, then change to Cer
mrm_list_offical['Class'] = mrm_list_offical['Class'].replace(['O','omega'], 'Cer')
#if TG then change to TAG
mrm_list_offical['Class'] = mrm_list_offical['Class'].replace('TG', 'TAG')
#make a column called Lipid Name match lipid column to lipid types
#If STD then use Compound Name as class
mrm_list_offical['Class'] = np.where(mrm_list_offical['Lipid'].str.contains('STD'), mrm_list_offical['Lipid'], mrm_list_offical['Class'])
#mrm_list_offical['Class'] = mrm_list_offical['Lipid'].str.extract('([A-Z]+)')
#make a column called Lipid Name match lipid column to lipid types
pd.set_option('display.max_rows', None)
print(mrm_list_offical.head(None))


In [ ]:
list_of_lipid_classes = mrm_list_offical['Class'].unique()
print(list_of_lipid_classes)

Load mzML file and convert to pandas dataframe and csv file. |
Columns = Q1, Q3, Intensity, Transition, Lipid, Class  |
Dataframes are stored in a dictionary called data_dict |
Key = mzml file name from /data_mzml folder |
Value = Pandas dataframe of data |
Parsed data is also stored as csv file in data_csv

In [ ]:
#Create for loop to load all mzml files from the data folder into the run object from pymzml reader function and store in pandas dataframe
#Create empty dictionary to store all the data
data_folder = os.listdir('./data_mzml/03-13-23/all/') #Path to the mzml files
path_to_mzml_files = './data_mzml/03-13-23/all/' #Path to the mzml files
data_dict = {} #Empty dictionary to store all the data

#Create name of pandas dataframe based off of the mzml file name
for file in data_folder:
    if file.endswith('.mzML'):
        print(file)
        run = pymzml.run.Reader(path_to_mzml_files+file, skip_chromatogram=False) #Load the mzml file into the run object
        print('Spectrum # = ',run.get_spectrum_count())
        print('Chromatogram # =',run.get_chromatogram_count())
        #create pandas dataframe to store the data with the columns Parent Ion, Product Ion, Intensity, Transition Lipid and Class
        df_sample = pd.DataFrame(columns=['Parent_Ion','Product_Ion','Intensity','Transition','Lipid','Class']) #Create empty pandas dataframe to store the data
        #df_sample = pd.DataFrame(columns=['Q1','Q3','Intensity','Transition','Lipid','Class']) #Create empty pandas dataframe to store the data
        q1_mz = 0 #Create empty variables to store the Q1 and Q3 m/z values
        q3_mz = 0
        count = 0 #Create a counter to keep track of the number of transitions
        for spectrum in run:
            for element in spectrum.ID.split(' '):
                    # print('element',element)
                    intensity_store = np.array([])
                    if 'Q1' in element:
                            #print('Q1',element)
                            q1 = element.split('=')
                            #print('q1',q1[1])
                            q1_mz= np.floor(round(float(q1[1]),1))
                            # print('q1',q1)
                    
                    if 'Q3' in element:
                            # print('Q3',element)
                            q3 = element.split('=')
                            #print('q3',q3[1])
                            q3_mz=np.floor(round(float(q3[1]),1))
                            # print('q3',q3)
                            # df_sample.loc[count,'Q1'] = q1_mz
                            # df_sample.loc[count,'Q3'] = q3_mz
                            
                            for mz,intensity in spectrum.peaks(): #Get the m/z and intensity values from the spectrum
                                    intensity_store = np.append(intensity_store,intensity) #Store the intensity values in an array
        
                    
                    if 'Q3' in element:
                        # print(intensity_sum)
                            intensity_sum = np.sum(intensity_store) #Sum the intensity values
                            df_sample.loc[count,'Parent_Ion'] = q1_mz #Store the Q1 and Q3 m/z values in the pandas dataframe
                            df_sample.loc[count,'Product_Ion'] = q3_mz
                            #round the Q1 and Q3 m/z values to 1 decimal places
                            df_sample.loc[count,'Parent_Ion'] = np.floor(round(df_sample.loc[count,'Parent_Ion'],1))
                            df_sample.loc[count,'Product_Ion'] = np.floor(round(df_sample.loc[count,'Product_Ion'],1))
                            df_sample.loc[count,'Intensity'] = intensity_sum #Store the intensity values in the pandas dataframe
                            df_sample.loc[count,'Transition'] = str(q1_mz)+ ' -> '+ str(q3_mz) #Store the transition values in the pandas dataframe
                            count+=1 
        data_dict[file] = df_sample #Store the pandas dataframe in the dictionary
        df_sample.to_csv('./data_csv/'+file+'.csv') #Save the data as a csv file in the data_csv folder
        print(df_sample.head())

print(data_dict.keys())

Confirm dictionary has properly stored all mzml files as key (mzml file name) and value (df of file contents). We can now see all mzml files from the data folder have been parsed into pandas dataframe and each df has been stored in a dictionary for easy access.

In [ ]:
#Print out key value pairs of the dictionary to confirm data is correct 
for key,value in data_dict.items():
    print(key)
    print(value.head())
    print(value)

#Sort keys in the dictionary
sorted_keys = sorted(data_dict.keys())
data_dict = {key:data_dict[key] for key in sorted_keys}
print(data_dict.keys())

Load MRM transitions from csv file to pandas dataframe. This list will be used to identify the possible lipids in our sample.

In [ ]:
# #list of MRM to pandas dataframe

# # Open Lipid File
# lipid_file_mrm = pd.read_csv('List_MRMs.csv', on_bad_lines='skip', sep='\t', header=None)
# #Create Headers for the dataframe
# headers = ['Lipid','Transition']
# lipid_file_mrm.columns = headers #Assign headers to the dataframe

# #split the transition column into Q1, Arrow, Q3
# start = lipid_file_mrm.Transition.str.split(expand=True) #Split the transition column into Q1, Arrow, Q3
# start.columns = ['Q1','Arrow','Q3'] #Assign the column names to the new dataframe
# transitions_headers = ['Q1','Arrow','Q3'] #Create a list of the column names

# #Create a new dataframe of mrm transitions with columns = Lipids and Q1, Q3
# df_mrm= pd.DataFrame(columns=['Lipid','Q1','Q3','Transition']) #Create a new dataframe of mrm transitions with columns = Lipids and Q1, Q3
# df_mrm['Lipid'] = lipid_file_mrm['Lipid'] #Assign the Lipid column from the lipid_file_mrm to the new dataframe
# df_mrm['Q1'] = start['Q1'] #    Assign the Q1 column from the start dataframe to the new dataframe
# df_mrm['Q3'] = start['Q3'] #    Assign the Q3 column from the start dataframe to the new dataframe
# #change the Q1 and Q3 columns to float
# df_mrm['Q1'] = df_mrm['Q1'].astype(float)
# df_mrm['Q3'] = df_mrm['Q3'].astype(float)
# #round the Q1 and Q3 m/z values to 1 decimal places
# df_mrm['Q1'] = np.floor(round(df_mrm['Q1'],1))
# df_mrm['Q3'] = np.floor(round(df_mrm['Q3'],1))
# #Change column name Q1 to Parent_Ion and Q3 to Product_Ion


# #For loop to create the transition column from the Q1 and Q3 columns
# for index in range(len(df_mrm)):
#     df_mrm.loc[index,'Transition'] = str(df_mrm.loc[index,'Q1']) + ' -> ' + str(df_mrm.loc[index,'Q3'])

# df_mrm.rename(columns={'Q1':'Parent_Ion'}, inplace=True)
# df_mrm.rename(columns={'Q3':'Product_Ion'}, inplace=True)
# #df_mrm['Transition'] = lipid_file_mrm['Transition'] #Assign the Transition column from the lipid_file_mrm to the new dataframe



# df_mrm.head()

In [ ]:
#Create dictionary to store matching transitions from df_mrm

matching_dic = {} #dictionary to store matching transitions
matching_dic2 = {}

# #loop through each sample dataframe in data_dict
# for key, value in data_dict.items():
#     #print(value.iloc[0:,:])
#     df_current = pd.DataFrame(columns=['Parent_Ion','Product_Ion','Intensity','Transition','Lipid','Class']) #create a new dataframe for each sample
#     #loop through each transition in the sample dataframe
#     for index in range(len(value)):
        
#         #Match Q1 and Q3 values in df_mrm to Q1 and Q3 values in value
#         for row in range(len(df_mrm)):
#             if df_mrm.loc[row,'Parent_Ion'] == value.iloc[index,0] and df_mrm.loc[row,'Product_Ion'] == value.iloc[index,1]:
#                 value.loc[index,'Lipid'] = df_mrm.loc[row,'Lipid']
#                 df_current = df_current.append(value.iloc[index,:]) #add the transition to the new dataframe
                

#     matching_dic[key] = df_current #add the new dataframe to the matching_dic dictionary

#do the same thing with mrm_list_offical and save to matching_dic2
for key, value in data_dict.items():
    #print(value.iloc[0:,:])
    df_current = pd.DataFrame(columns=['Parent_Ion','Product_Ion','Intensity','Transition','Lipid','Class']) #create a new dataframe for each sample
    #loop through each transition in the sample dataframe
    for index in range(len(value)):
        
        #Match Q1 and Q3 values in df_mrm to Q1 and Q3 values in value
        for row in range(len(mrm_list_offical)):
            if mrm_list_offical.loc[row,'Parent_Ion'] == value.iloc[index,0] and mrm_list_offical.loc[row,'Product_Ion'] == value.iloc[index,1]:
                value.loc[index,'Lipid'] = mrm_list_offical.loc[row,'Lipid']
                value.loc[index,'Class'] = mrm_list_offical.loc[row,'Class']
                df_current = df_current.append(value.iloc[index,:]) #add the transition to the new dataframe
                

    matching_dic[key] = df_current #add the new dataframe to the matching_dic dictionary

In [ ]:
print(mrm_list_offical.head())

In [ ]:
# for key, value in matching_dic.items():
#     if value.empty:
#         print(key, 'is empty')
#     else:
#         print(key)
#         print(value.head())
#         print('_____________________________________________________________')

In [ ]:
# for key, value in matching_dic.items():
#     if value.empty:
#         print(key, 'is empty')
#     else:
#         print(key)
#         print(value.head())
#         print(value)
#         print('_____________________________________________________________')

In [ ]:
#Save each dataframe from the matching_dic dictionary to an excel file in the data_results/data/data_matching folder
# for key, value in matching_dic.items():
#     if value.empty:
#         print(key, 'is empty')
#     else:
#         value.to_excel('data_results/data/data_matching/' + key + '.xlsx', index=False)
#         print(key, 'saved')
#         print('_____________________________________________________________')

In [ ]:
# # print(data_dict['test_sample'])
# # print(matching_dic['test_sample'])

# #iterate through each dataframe in matching dict and add the lipid and class to the dataframe from df_mrm
# for key, value in matching_dic.items():
#     for index in range(len(value)): #iterate through each row in the dataframe
#         lipid = df_mrm.loc[df_mrm['Transition'] == value.iloc[index,3], 'Lipid'].iloc[0] #get the lipid from df_mrm
#         matching_dic[key].loc[index,'Lipid'] = lipid #add the lipid to the dataframe


# #print(matching_dic['test_sample'])
# print(matching_dic)

In [ ]:
# # #Create for loop to store all processed dataframes with intensity > 20% max intensity in data_results/data folder
# # for key, value in data_dict.items():
# #     if value.empty:
# #         continue
# #     else:
# #         value.to_csv('./data_results/data/data_filtered/'+key+'.csv') #Save the data as a csv file
# #         print(value.head())

# for key, value in matching_dic.items():
#     if value.empty:
#         continue
#     else:
#         value.to_csv('./data_results/data/data_matching/fly_03_08_23/TAG/'+key+'.csv') #Save the data as a csv file
#         print(value.head())


In [ ]:
#import visualization libraries
import umap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# #Plotting Transition vs Intensity for each sample in matching_dic with color = Lipid

# for key,value in matching_dic.items():
#     if value.empty:
#         print(key, 'is empty')
#         pass
#     else:
#         fig = px.bar(value,title=key, x='Transition', y='Intensity',color='Lipid')
#         fig.show()

# # fig = px.bar(matching_dic['test_sample'],title='test_sample', x='Transition', y='Intensity', color='Lipid')
# # fig.show()

In [ ]:
#create a for loop to store all the plots in data_results/plots/plots_matching folder
# folder_name_plots = '03-13-23'
# for key, value in matching_dic.items():
#     if value.empty:
#         continue
#     else:
#         fig = px.bar(value,title=key, x='Transition', y='Intensity',color='Lipid')
#         fig.show()
#         fig.write_image("./data_results/plots/plots_matching/{}"+key+".png".format(folder_name_plots))
#         #fig.write_image("./data_results/plots/plots_matching/"+key+".pdf")
#         print('_____________________________________________________________')

In [ ]:
################
#BLANK SUBTRACTION
################
# #data_dict['TAG1_Blank.mzML'].head()
# #Create for loop and subtract the blank intensity from the sample intensity
# data_dict_blanks_subtracted = {}
# for key,value in data_dict.items():
#     if 'Blank' in key:
#         #print(key)
#         blank = value #Store the blank data in a variable
# for key,value in data_dict.items():
#     if 'Blank' not in key:
#         #print(key)
#         sample = value #Store the sample data in a variable
#         sample['Intensity'] = sample['Intensity'] - blank['Intensity'] #Subtract the blank intensity from the sample intensity
#         data_dict_blanks_subtracted[key] = sample #Store the new sample data in the dictionary
#         #plot transition intensities
#         # fig = px.bar(sample, x="Transition", y="Intensity", title=key)
#         # fig.show()

# #create the same for loop but remove the negative values
# for key,value in data_dict.items():
#     if 'Blank' not in key:
#         print(key)
#         #sample = value #Store the sample data in a variable
#         #sample['Intensity'] = sample['Intensity'] - blank['Intensity'] #Subtract the blank intensity from the sample intensity
#         #Remove the negative values
#         sample['Intensity'] = sample['Intensity'].apply(lambda x: 0 if x < 0 else x)


#         #plot transition intensities
#         # for i in range(len(value)):
#         #     if value['Intensity'][i] < 0.2*max(value['Intensity']):
#         #         value.drop(i,inplace=True)
#         # fig = px.bar(sample, x="Transition", y="Intensity",color='Lipid' ,title=key)
#         fig.show()

In [ ]:
#create a new df that subtracts the TAG1_BlankN1 from each sample
#subtract intensity column of the blank from each sample
# data_dict_blanks_subtracted = {}
# #print(data_dict_blanks_subtracted)
# for key, value in data_dict.items():
#     #data_dict_blanks_subtracted[key] = data_dict[key].subtract(data_dict['TAG1_Blank_N1_03082023.mzML'], axis=1) #subtract the blank from each sample
# #     for i in range(len(value)):
# #         if value['Intensity'][i] < 0.2*max(value['Intensity']):
# #                 value.drop(i,inplace=True)
#         fig = px.bar(value,title=key, x='Transition',y='Intensity',color='Lipid',range_y= [0.2*max(value['Intensity']),max(value['Intensity'])])
#         fig.show()
#         # print(key)

# for key, value in data_dict.items():
#     print(key)

In [ ]:
#Plot intensity vs class for each sample
# for key, value in data_dict.items():
#     if value.empty:
#         continue
#     else:
#         fig = px.bar(value,title=key, x='Class', y='Intensity',color='Lipid')
#         fig.show()
#         print('_____________________________________________________________')

In [ ]:
# #make a pie chart of intensity vs class for each sample
# for key, value in data_dict.items():
#     if value.empty:
#         continue
#     else:
#         fig = px.pie(value, values='Intensity', names='Class', title=key)
#         fig.show()
#         print('_____________________________________________________________')

#make a pie chart of intensity vs lipid for each sample
# for key, value in data_dict.items():
#     if value.empty:
#         continue
#     else:
#         fig = px.pie(value, values='Intensity', names='Lipid', title=key)
#         fig.show()
#         print('_____________________________________________________________')